In [1]:
from __future__ import print_function,division
import numpy as np
import nltk
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import cPickle as pickle

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
reviews = pd.read_csv("ConciseReviews.csv", sep=None,)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [3]:
# Inspecting some of the reviews
for i in range(5):
    print("Review #",i+1)
    print("Summary: ",reviews.TargetSummary[i],"\n")
    print("Reviews: ",reviews.ConciseMovieReview[i],"\n\n")
    print()

Review # 1
Summary:  a powerful documentarylike examination of the response to an occupying force the battle of algiers has not aged a bit since its release in 1966 

Reviews:  a riveting reconstruction of the struggle for independence in mid-50s algiers, its pseudo-documentary style still feels as headline-fresh as its content. brilliantly directed set-pieces and remarkable thronging crowd scenes make the film a masterpiece; the ominous familiarity of its subject makes it a must-see. gripping, relevant cinemairish film, "bloody sunday," borrowed its entire structure from "algiers". "the battle of algiers" is a one of a kind masterpiece of pure cinema that you will never forget. the film establishes a kinetic documentary effect, making the impact of every shoot-out and explosion a deeply personal experience. this seminal meticulously crafted work about algiers struggle for freedom won the venic film fest top award, and deservedly garnered pontecorvo oscar nominations as best director a

In [4]:
reviews.TargetSummary[182]

'thanks to energetic performances from its young leads gimme the loot captures a slice of city life with warmth and exuberance'

In [5]:
SampledReviews=[]
Summaries=[]
ts=[]
for i in range(2200):
    SampledReviews.append(reviews.ConciseMovieReview[i])
    ts.append(reviews.TargetSummary[i])
    Summaries.append('SOS '+reviews.TargetSummary[i]+' EOS')

In [6]:
Summaries[:3]

['SOS a powerful documentarylike examination of the response to an occupying force the battle of algiers has not aged a bit since its release in 1966 EOS',
 'SOS poor plot development and slow pacing keep 54 from capturing the energy of it is legendary namesake EOS',
 'SOS while it hews closely to the 1984 original craig brewer infuses his footloose remake with toetapping energy and manages to keep the story fresh for a new generation EOS']

In [7]:
extractedtokenizer=Tokenizer(char_level=False)
extractedtokenizer.fit_on_texts(SampledReviews)

In [8]:
extractedWord_counts=extractedtokenizer.word_counts

In [9]:
extractedWord_counts

OrderedDict([('a', 43488),
             ('riveting', 195),
             ('reconstruction', 5),
             ('of', 38058),
             ('the', 65052),
             ('struggle', 70),
             ('for', 8813),
             ('independence', 25),
             ('in', 17394),
             ('mid', 52),
             ('50s', 39),
             ('algiers', 6),
             ('its', 7465),
             ('pseudo', 23),
             ('documentary', 991),
             ('style', 691),
             ('still', 1211),
             ('feels', 946),
             ('as', 10989),
             ('headline', 8),
             ('fresh', 314),
             ('content', 98),
             ('brilliantly', 182),
             ('directed', 575),
             ('set', 669),
             ('pieces', 173),
             ('and', 39149),
             ('remarkable', 216),
             ('thronging', 1),
             ('crowd', 166),
             ('scenes', 620),
             ('make', 1809),
             ('film', 10338),
            

In [10]:
print('No of unique words in extracted Reviews:->',len(extractedWord_counts))

No of unique words in extracted Reviews:-> 39118


In [11]:
summarytokenizer=Tokenizer(char_level=False)
summarytokenizer.fit_on_texts(ts)

In [12]:
summaryWord_counts=summarytokenizer.word_counts
summaryWord_index=summarytokenizer.word_index

In [13]:
summaryWord_index['the']

1

In [14]:
len(summarytokenizer.word_counts)

8527

In [15]:
modiefiedSummaryWord_index=summarytokenizer.word_index
print('Initially vocab size for modified summary is:->',len(modiefiedSummaryWord_index))
modiefiedSummaryWord_index['PAD']=0
print('After adding PAD to vocab, vocab size for modified summary is:->',len(modiefiedSummaryWord_index))
modiefiedSummaryWord_index['SOS']=len(summarytokenizer.word_counts)+1
print('After adding SOS to vocab, vocab size for modified summary is:->',len(modiefiedSummaryWord_index))
modiefiedSummaryWord_index['EOS']=len(summarytokenizer.word_counts)+2
print('After adding EOS to vocab, vocab size for modified summary is:->',len(modiefiedSummaryWord_index))


Initially vocab size for modified summary is:-> 8527
After adding PAD to vocab, vocab size for modified summary is:-> 8528
After adding SOS to vocab, vocab size for modified summary is:-> 8529
After adding EOS to vocab, vocab size for modified summary is:-> 8530


In [16]:
summaryWord_index['battle'],modiefiedSummaryWord_index['battle']

(747, 747)

In [17]:
embeddings = dict()
filename = 'glove.6B/glove.6B.100d.txt'
f = open(filename)
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings))

Loaded 400000 word vectors.


In [18]:
print('No of unique words in summaries:->',len(summarytokenizer.word_counts))

No of unique words in summaries:-> 8527


In [19]:
ReviewsVocabSize=len(extractedtokenizer.word_index)+1
SummaryVocabSize=len(summarytokenizer.word_index)+1#actually it is 11279
ModifiedVocabSize=len(modiefiedSummaryWord_index)+1
print('ReviewsVocabSize:->',ReviewsVocabSize)
print('SummaryVocabSize:->',SummaryVocabSize)
print('ModifiedVocabSize:->',ModifiedVocabSize)

ReviewsVocabSize:-> 39119
SummaryVocabSize:-> 8531
ModifiedVocabSize:-> 8531


In [20]:
summaryWord_index['SOS']

8528

In [21]:
modiefiedSummaryWord_index['SOS']

8528

In [22]:
extractedtokenizer.word_index

{'woods': 6262,
 'clotted': 38260,
 'hanging': 5717,
 'woody': 1089,
 'comically': 12345,
 'spidery': 37423,
 'hermans': 19225,
 'aileen': 7418,
 'scold': 16872,
 'originality': 1398,
 'rickman': 14470,
 'rumbustious': 33769,
 'wracked': 16480,
 'appropriation': 13323,
 'bitebut': 35498,
 'compuesta': 37082,
 'bringing': 1374,
 'wooded': 25420,
 'liaisons': 21092,
 'grueling': 6777,
 'wooden': 6242,
 'wednesday': 31068,
 'himselves': 35113,
 'circuitry': 21245,
 'crotch': 13992,
 'stereotypical': 9993,
 'steadfastness': 34689,
 'gwenns': 26999,
 'insular': 12578,
 'scrapes': 22842,
 'precocity': 25421,
 '274': 35619,
 'imploding': 35808,
 'sustaining': 16868,
 'consenting': 28587,
 'scraped': 25653,
 'saddams': 32367,
 'inanimate': 20425,
 'dialogu': 37792,
 'errors': 15291,
 'deferred': 32838,
 'baddie': 35734,
 'cooking': 10692,
 'opportunists': 35323,
 'videodrome': 11211,
 'numeral': 34677,
 'succumb': 9108,
 'shocks': 4518,
 'brainwashed': 29481,
 '1997s': 37157,
 'ching': 35404,


In [23]:
embeddingReviews = np.zeros((ReviewsVocabSize, 100))
embedding_dim=100
for word, i in extractedtokenizer.word_index.items():
    embedding_vector = embeddings.get(word)
    if embedding_vector is not None:
        embeddingReviews[i] = embedding_vector
    else:
        new_embedding = np.array(np.random.uniform(-1.0, 1.0, embedding_dim))
        embeddings[word]=new_embedding
        embeddingReviews[i]=new_embedding 

In [24]:
len(embeddingReviews)

39119

In [25]:
FN1 = 'embeddingReviewsFilewithOtherdata'
FN0='embeddingReviewsFile'

In [26]:
import cPickle as pickle

In [27]:
with open('%s.pkl'%FN0,'wb') as fp:
    pickle.dump((embeddingReviews),fp,-1)

In [28]:
encodedReviews=extractedtokenizer.texts_to_sequences(SampledReviews)
encodedSummaries=summarytokenizer.texts_to_sequences(Summaries)
#encodedTargetSummaries=ts.texts_to_sequences

In [29]:
import psutil

In [30]:
psutil.virtual_memory()

svmem(total=8240283648, available=4650946560, percent=43.6, used=3084795904, free=2868744192, active=3830026240, inactive=1276313600, buffers=632348672, cached=1654394880, shared=239992832)

In [31]:
encoded_modified_summaries=[]
count=0
for line in Summaries:
    wordList=[]
    count+=1
    #print('count:->',count)
    for word in nltk.word_tokenize(line):
        #print(word)
        if modiefiedSummaryWord_index.has_key(word) !=False:
            wordList.append(modiefiedSummaryWord_index[word])
    encoded_modified_summaries.append(wordList)


In [32]:
print(encoded_modified_summaries[0])

[8528, 3, 79, 3774, 498, 4, 1, 3775, 6, 12, 3776, 1154, 1, 747, 4, 3777, 37, 19, 3778, 3, 385, 1421, 9, 2456, 8, 3779, 8529]


In [33]:
len(encodedSummaries)

2200

In [34]:
testEncodedSummaries=encodedSummaries[2000:]#950
testEncoded_Reviews=encodedReviews[2000:]
#testEncoded_modiefiedSummaries=encoded_modified_summaries[3600:]

In [35]:
encodedReviews=encodedReviews[:2000]
encodedSummaries=encodedSummaries[:2000]
encoded_modified_summaries=encoded_modified_summaries[:2000]

In [36]:
len(testEncoded_Reviews),len(testEncodedSummaries)#,len(testEncoded_modiefiedSummaries)

(200, 200)

In [37]:
print(encodedSummaries[0])

[3, 79, 3774, 498, 4, 1, 3775, 6, 12, 3776, 1154, 1, 747, 4, 3777, 37, 19, 3778, 3, 385, 1421, 9, 2456, 8, 3779]


In [38]:
mydata={'encodedReviews':encodedReviews,'encodedSummaries':encodedSummaries,'encoded_modified_summaries':encoded_modified_summaries,'testEncoded_Reviews':testEncoded_Reviews,'testEncodedSummaries':testEncodedSummaries}
        #'valEncodedReviews':valEncodedReviews,'valEncodedSummaries':valEncodedSummaries,'valEncodedModifiedSummaries':valEncodedModifiedSummaries}

In [39]:
with open('%s.pkl'%FN1,'wb') as fp:
    pickle.dump((mydata),fp,-1)

In [40]:
psutil.virtual_memory()

svmem(total=8240283648, available=4644876288, percent=43.6, used=3090690048, free=2857709568, active=3836407808, inactive=1280831488, buffers=635764736, cached=1656119296, shared=239988736)

In [41]:
maxReviewLength=300
maxSummaryLength=40

In [42]:
paddedReviews = pad_sequences(encodedReviews, maxlen=maxReviewLength, padding='post',truncating='post')
paddedSummary = pad_sequences(encodedSummaries, maxlen=maxSummaryLength, padding='post',truncating='post')
paddedModifiedSummary = pad_sequences(encoded_modified_summaries, maxlen=maxSummaryLength, padding='post',truncating='post')

In [43]:
testPaddedReviews = pad_sequences(testEncoded_Reviews, maxlen=maxReviewLength, padding='post',truncating='post')
testPaddedSummary = pad_sequences(testEncodedSummaries, maxlen=maxSummaryLength, padding='post',truncating='post')
#testPaddedModifiedSummary = pad_sequences(testEncoded_modiefiedSummaries, maxlen=maxSummaryLength, padding='post',truncating='post')

In [44]:
myPaddedData={'paddedReviews':paddedReviews,'paddedSummary':paddedSummary,'paddedModifiedSummary':paddedModifiedSummary,'testPaddedReviews':testPaddedReviews,'testPaddedSummary':testPaddedSummary}
              #,'valPaddedReviews':valPaddedReviews,'valPaddedSummary':valPaddedSummary}

In [45]:
with open('%s.pkl'%'myPaddedDataFile','wb') as fp:
    pickle.dump((embeddingReviews,modiefiedSummaryWord_index,myPaddedData),fp,-1)

In [46]:
print(paddedReviews[1240])

[ 4267  8451    16    15  5313    20     2  2737     4   812  1452   149
   936    30   184    81     3    20     2   203   251   933   253  1522
 23673    47   287    19 13391 32850 32851    29   296  3680    94    56
     7    10   123  2228     1  2229     4 19260  2500   586   993     7
     2    12    30     1   730    41  4107 17190  1288    65     5    17
    59     6    81    30  8751     1  2364   149  1162 23674 10003     4
     2  1412  6906  2998     5    17  2868    21    37  9362  9940  4715
   582  9362  9940  4215    45  8751     5   270   932     7   391     8
     5 14980  1013    36   121     1   340   211    18  1858    42 23407
   356   103  8751     5     2  4473 12954 12954     2  3193   652  1752
     9 10395  3155  1930     3     2   127     9  4379 17505    49  1930
     7    81     8  3424   760     4     2   895   170     6  4881     1
   130     4     1  2104  5738  8751    50     2  1485   140    45     1
    15  8751   993  4079  3101    17    80 32852   

In [47]:
print(paddedSummary[23])

[3837   20    1  851  852   98 1828 2488    2   47 3838 2489    3  980
  280   18  131 1160 1161 2490 2491 2492    5    3  148    2 1829  679
   44    0    0    0    0    0    0    0    0    0    0    0]


In [48]:
print(paddedModifiedSummary[0])

[8528    3   79 3774  498    4    1 3775    6   12 3776 1154    1  747
    4 3777   37   19 3778    3  385 1421    9 2456    8 3779 8529    0
    0    0    0    0    0    0    0    0    0    0    0    0]
